## Part 2: MCMC Diagnostic

在前面我们已经了解到，如何通过概率统计语言 PyMC3 来帮我们实施 MCMC 算法。
```python
with model:
    trace = pm.sample(draws = 2000, chains=2)
```
比如在上述代码示例中，我们对贝叶斯模型采样2000个样本，并同时运行2条马尔科夫链。

我们凭借这2000个样本来代表我们的目标分布(后验分布)。

**而问题在于：**
- 这2000个样本真的能代表目标分布吗？
- 我们如何检测MCMC的样本是否存在问题？

注意：
- PyMC 的结果 trace 中包含多个 MCMC chain
- 每个 chain 中可能包括多个参数
- 每个参数的采样都可以代表为一个参数后验分布(目标分布)

为了方便，接来的讨论的**单个chain**都只包含**单个参数**。

### 什么是 MCMC diagnostic?

为了检测 MCMC 样本的质量与问题，我们需要一些诊断工具，这些诊断工具统称为：马尔可夫链蒙特卡罗诊断(Markov Chain Monte Carlo diagnostics)，简称 MCMC diagnostic。

MCMC diagnostic 可以用来检查用MCMC算法采样得到的样本是否足以提供目标分布的精确近似。

MCMC diagnostic 的目的：
1. 发现 MCMC 样本存在的问题。
2. 根据诊断对 MCMC 算法进行修正(fix)。

MCMC diagnostic 检查的具体内容:
- MCMC样本的大部分采样是否来自目标分布(target distribution)。
- MCMC样本量是否足够近似目标分布。

### MCMC 样本的两大问题

要了解 MCMC diagnostic 检测到底是什么，首先我们需要知道 MCMC 样本最常见的问题。

MCMC 样本最常见的两大问题：
1. bias: 样本是否代表整个后验分布?
2. precision: 是否有足够的样本来获得足够精确的统计推断?

这两个问题涉及到 MCMC 诊断的两个专有概念：收敛性与有效样本量。

1. 收敛(convergence)问题。
   
   由于马尔科夫链的性质，无论 MCMC 算法的起始值(initial or starting value)和起始分布为何，随着采样的进行，样本最终都应该接近目标分布。

   而由于各种问题(比如，代码错误，或者先验定义错误)，样本无法接近目标分布，我们称为 MCMC chain **无法收敛**。见下图2。
   
   即使 MCMC chain 可以收敛，如果起始值离目标分布太远，那么它收敛所需的时间也更长，见下图3。

2. 有效样本量(effective sample size)过于少的问题。

   可以想象，当样本量越多，样本分布越接近目标分布，但是当样本数量过少时，可能对目标分布产生错误的推断。那什么因素会导致采样样本的数量变少？
   
   由于马尔科夫链的性质，当前采样 $\theta_{t}$ 依赖于 上一次的采样 $\theta_{t-1}$。因此，样本间的关系不是独立的，这违反了独立采样的假设。违反独立采样的后果可能就是 MCMC chain 无法收敛，如下图2。

   可以想象，如果 $\theta_{t}$ 与 $\theta_{t-n}$ 的间隔 n 越大，两个样本的相关性越低。

   **通过每隔 n 保留一个采样可以减少样本相关性的影响，但代价就是有效样本量变少**。

图1：参数收敛时MCMC采样的样本


![Image Name](https://cdn.kesci.com/upload/image/rkijyv4px4.png?imageView2/0/w/300/h/640)


图2：参数不收敛时MCMC采样的样本

![Image Name](https://cdn.kesci.com/upload/image/rkijynzzao.png?imageView2/0/w/300/h/640)

图3：起始值过于远离目标分布时的MCMC采样

![Image Name](https://cdn.kesci.com/upload/image/rkijy7xre5.png?imageView2/0/w/300/h/640)

source: https://statlect.com/fundamentals-of-statistics/Markov-Chain-Monte-Carlo-diagnostics

### MCMC 视觉检查 (Visual Diagnostics)

检查 MCMC 采样质量最直观的方式就是视觉检查 (Visual Diagnostics)。

上面的采样轨迹图就是视觉检查的一种形式。所以我们对此并不陌生。

常见的视觉检查有两类：
- 轨迹图 Trace plots。即上面的图。
- 自相关函数图 Autocorrelation function (ACF) plots。
  
  前面提到样本间存在相关性，而这种自相关性会导致轨迹图“不收敛”。而不收敛的情况有很多，并不能从轨迹图中得知是因为自相关导致了不收敛，因此我们需要自相关函数图来诊断是否存在自相关的问题。

接下来我们结合代码示例进行演示。

以前使用过的 `arviz` 工具包就可以完成大部分的诊断工作。所以接下来的分析我们会依赖于 arviz。

In [10]:
#基本的库和模块
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt

#贝叶斯模型探索分析
import arviz as az

# 概率编程语言
import pymc3 as pm

为了演示上述不同问题对于 MCMC 采样的影响，我们生成了三种 MCMC chain:

- good_chains 包含两条 MCMC chain，生成自一个 $\beta$ 分布，并且两次采样间是完全独立的。
- bad_chains0 在 good_chains 的基础上，加入了采样间的自相关。
- bad_chains1 在 good_chains 的基础上，在采样的局部加入高度自相关。bad_chains1 代表了一个非常常见的场景，采样器可以很好地解析参数空间的一个区域，但是很难对一个或多个区域进行采样。


> 代码源自：2.4 Diagnosing Numerical Inference in Martin, from O. A., Kumar, R., & Lao, J. (2021). Bayesian Modeling and Computation in Python (1st ed.). Chapman and Hall/CRC. https://doi.org/10.1201/9781003019169


In [11]:
good_chains = stats.beta.rvs(2, 5,size=(2, 2000))
bad_chains0 = np.random.normal(np.sort(good_chains, axis=None), 0.05, size=4000).reshape(2, -1)
bad_chains1 = good_chains.copy()
for i in np.random.randint(1900, size=4):
  bad_chains1[i%2:,i:i+100] = np.random.beta(i, 950, size=100)

chains = {"good_chains":good_chains,
          "bad_chains0":bad_chains0,
          "bad_chains1":bad_chains1}

#### Trace Plots

轨迹图(trace plot)是贝叶斯中最常见的诊断图，横坐标为每个迭代步骤，纵坐标为采样值。

从这些图中，我们能够看到 
- MCMC chain 是否收敛。
- 同一个参数每条链是否收敛为相同的分布。
- 初步观察自相关程度。
  
在ArviZ中，通过调用函数az.plot_trace(.)，可以在右侧得到轨迹图，在左侧得到参数分布图。

In [12]:
az.plot_trace(chains)
plt.show()

<Figure size 864x432 with 6 Axes>

可以看到，
- good_chains 的表现最好，并且其中的两条 chain 几乎重合。
- bad_chains0 并不收敛，其中的两条 chain 并不趋向于同一个目标分布，并且两者有持续扩大的趋势。
- bad_chains1 虽然收敛，但是采样不均匀，即多个波峰(左图)，表现为在局部过多采样(右图)。

此外，如前面提到的，对于多条 MCMC 链，trace plot 还可以观测到不同初始值对采样的影响。

![](https://bcss.org.my/tut/wp-content/uploads/2020/04/rw3.gif)

source: https://bcss.org.my/tut/bayes-with-jags-a-tutorial-for-wildlife-researchers/mcmc-and-the-guts-of-jags/mcmc-diagnostics/

比如，由于初始值过于极端，导致需要花费多次迭代才能达到平稳分布。

![Image Name](https://cdn.kesci.com/upload/image/rkijy7xre5.png?imageView2/0/w/400/h/640)

#### Autocorrelation Plots

正如我们在讨论有效样本容量时所看到的，自相关减少了样本中包含的样本量。

因此我们希望自相关尽量小的同时样本量足够多。

我们可以用`az.plot_autocorr`绘制自相关图(autocorrelation plot)检查自相关性。

In [13]:
az.plot_autocorr(chains, combined=True) # combined=True表明将两条chains合并为1条
plt.show()

<Figure size 1490.4x331.2 with 3 Axes>

上图为自相关函数在100步间隔窗口上的条形图。
- 横坐标为 $\theta_{t}$ 与 $\theta_{t+n}$ 的间隔 n，该间隔越大，两个样本的相关性越低
- 纵轴为 $\theta_{t}$ 与 $\theta_{t+n}$ 的相关性。
- 灰色带表示所有相关系数形成的95%置信区间。

该图结果显示：
- good_chains 的纵向高度接近于零(并且大部分在灰色带内)，这表明自相关性非常低。
- bad_chains0 和 bad_chains1 中的高条表示自相关性值较大。
- bad_chains1 随着 n 的间隔增大，自相关减少。而 bad_chains0 不会随着 n 增大减少，说明该 MCMC chain的自相关问题非常严重。

为了更好的理解自相关系数(y轴)如何计算，以及 间隔n对自相关的影响。

我们可以自定义一个函数：
- 首先，取出 $\theta_{t}$
- 然后，取出 $\theta_{t+n}$
- 最后计算两者的相关系数

需要注意，在当前的例子中，t的总长度为2000，每当 n 增加1，$\theta_{t}$ 和 $\theta_{t+n}$ 的长度就会减少1。因为间隔不可能超过2000。

In [14]:
def autocorr(lag, chain):
    n = lag                                                 # 定义间隔n的大小
    t = 2000                                                # 单条 chain的总长度
    theta_t = chain[0:(t-1-n)]                              # 提取 theta_t
    theta_tn = chain[n:(t-1)]                               # 提取 theta_t+n
    rho = np.corrcoef(theta_t,theta_tn)                     # 计算相关系数
    print("间隔为", n, "时，的相关系数", rho.round(2)[0][1])   # 输出结果

In [15]:
# 间隔 n = 1时，三条链的自相关性
n = 1
autocorr(n, chains["good_chains"][0])
autocorr(n, chains["bad_chains0"][0])
autocorr(n, chains["bad_chains1"][0])

间隔为 1 时，的相关系数 0.01
间隔为 1 时，的相关系数 0.64
间隔为 1 时，的相关系数 0.15


In [16]:
# 间隔 n = 2时，三条链的自相关性
n = 2
autocorr(n, chains["good_chains"][0])
autocorr(n, chains["bad_chains0"][0])
autocorr(n, chains["bad_chains1"][0])

间隔为 2 时，的相关系数 0.02
间隔为 2 时，的相关系数 0.65
间隔为 2 时，的相关系数 0.18


In [17]:
# 间隔 n = 3时，三条链的自相关性
n = 3
autocorr(n, chains["good_chains"][0])
autocorr(n, chains["bad_chains0"][0])
autocorr(n, chains["bad_chains1"][0])

间隔为 3 时，的相关系数 0.0
间隔为 3 时，的相关系数 0.63
间隔为 3 时，的相关系数 0.16


In [18]:
# 间隔 n = 50时，三条链的自相关性
n = 50
autocorr(n, chains["good_chains"][0])
autocorr(n, chains["bad_chains0"][0])
autocorr(n, chains["bad_chains1"][0])

间隔为 50 时，的相关系数 0.0
间隔为 50 时，的相关系数 0.63
间隔为 50 时，的相关系数 0.08


可以看到，结果与 autocorrelation的图结果一致：
- good_chains 的结果始终在0附近，表明自相关低。
- bad_chains0 的结果显示，当间隔n扩大到50时，自相关系数从0.64下降到0.63。
- bad_chains1 的结果显示，当间隔n扩大到50时，自相关系数从0.15下降到0.08附近。

上述计算过程可以通过 Arviz自带的 `autocorr`函数进行计算。

In [29]:
n = [1,2,3,50]  # 定义间隔为 1，2，3 和 50
rhos = az.autocorr(chains["good_chains"][0]).round(2)[n]
print("good_chains的相关系数：",rhos)
rhos = az.autocorr(chains["bad_chains0"][0]).round(2)[n]
print("good_chains的相关系数：",rhos)
rhos = az.autocorr(chains["bad_chains1"][0]).round(2)[n]
print("good_chains的相关系数：",rhos)

good_chains的相关系数： [0.01 0.02 0.   0.  ]
good_chains的相关系数： [0.64 0.64 0.63 0.59]
good_chains的相关系数： [0.15 0.18 0.16 0.08]


通过轨迹图和自相关图，我们可以诊断出 MCMC chain 中存在的不同问题。

但自然而然能想到的两个问题是：
- 通过视觉方法判断 MCMC 的问题是否主观？能否通过客观的指标进行诊断？
- 发现了  MCMC 的问题，我们如何进行修正？

### MCMC 诊断指标

通过绘图对MCMC进行检查的优势在于直观，并且能通过图推测问题在什么地方。

但对于许多参数的模型来说，挨个检查图形是非常困难的。

因此，通过数值进行判断一方面可以提高诊断的客观性，另一方面提高了诊断的效率。

针对 MCMC 的两大问题，这里存在两个主要的指标：
1. 针对收敛问题的：Rhat，$\hat{R}$，也称为“潜在规模缩减因子” (Potential Scale Reduction Factor, PSRF)。它将单个链样本的变异与混合了所有链样本的变异进行比较。
2. 针对有效样本量问题的：effective sample size (ESS) or effective number of draws (n.eff)。

#### Convergence and Rhat


前面提到，收敛(convergence)描述了 MCMC 样本两个层面的问题：
1. MCMC 采样是否从初始值“收敛”到一个稳定的分布。
2. 即使链收敛到了一个稳定的分布，多条链形成的稳定分布是否相似。

反面例子就是 bad_chains0: 

该采样既没有收敛到一个稳定的分布(右图中采样整体向上偏移)，并且两个链形成的分布也不相同(左图)。
![Image Name](https://cdn.kesci.com/upload/image/rkimd3udb3.png?imageView2/0/w/640/h/640)


Rhat的作用就是通过一个指标反应上述两个层面的问题。

具体逻辑为：
- 运行多条 MCMC 链，并且以不同的初始值开始采样。
- 比较每条链的相似性。

数学逻辑为：
- 计算θ的所有样本的标准差，包括所有链的样本
- 计算每条链标准偏差，并通过平方平均数整合到一起
- 比较两个标准差的大小。如果两个的除数为1，说明两个变异相同，表明他们存在相似性。
- 如果，该值大于1.01，说明两个变异可能存在差异，即参数可能不收敛。

source: 
- https://www.r-bloggers.com/2020/01/applied-bayesian-statistics-using-stan-and-r/

假设，参数 $\theta$ 有m个 chain 和 n个 样本。

1. 用 $s_m^2$ 表示每条链内部的方差。那么链内部的变异 within-chain variance $W$ 为所有链方差的均值 $W = \frac{1}{M} \, \sum_{m=1}^M s_m^2$
2. 用 $\bar{s}_m^2$ 表示所有链均值计算的方差。链间的变异 between-chain variance $B$ 为 $B = \bar{s}_m^2$。
3. 结合在一起得到 Rhat $\hat{R} = \sqrt{\frac{W+B}{W}}$

可见，原理和方差分析类似，如果链间变异B趋近于0，那么Rhat趋近于1，说明各链的均值几乎相同，以此反应他们的相似性。

这里的数学公式有所简化，详细请看 https://mc-stan.org/docs/reference-manual/notation-for-samples-chains-and-draws.html。

Rhat计算的条件是有多条 MCMC 链并且尽可能的每条链使用不同的起始值。

我们可以通过 `az.rhat(.)` 简化计算过程。

In [23]:
az.rhat(chains)

<xarray.Dataset>
Dimensions:      ()
Data variables:
    good_chains  float64 1.001
    bad_chains0  float64 2.435
    bad_chains1  float64 1.018

从这个结果中我们可以看出，
- good_chains 的 Rhat 接近1，表示该参数的多条链都是收敛并且相似的。
- bad_chains0 的结果非常糟糕，这可以结合轨迹图得到佐证。
- bad_chains1 的结果相对较好，但其Rhat仍然大于1.01，说明该参数的收敛程度差，这点通过轨迹图有时难以发现其存在的问题。

Rhat的最低要求是低于1.1，但现在大多数发表的文章要求小于1.01。

#### 有效样本量 effective sample size (ESS) 与自相关 autocorrelation

前面通过自相关函数图发现，bad_chains1 会随着 n 间隔增大而自相关减少。

但这样做的代价是可用的样本数量变少。


![Image Name](https://cdn.kesci.com/upload/image/rkimhrvf7h.png?imageView2/0/w/640/h/640)


因此，我们是否能通过一个指标反应，在满足最小自相关的条件下，即只保留隔n个采样的样本时，可用的样本还剩下多少？

这就是 有效样本量(effective sample size,ESS)指标的意义。

ESS计算的逻辑为：
- 在样本量为n的 MCMC链中，分别计算 1到n 采样间隔下的样本相关性 $\rho$，这样可以计算得到，n个 $\rho_{n}$。
- 如果样本间自相关越大，$\rho$ 越接近1，相反则接近0。因此，n个 $\rho_{n}$ 相加，其值越接近n，说明自相关越大。
- 结合这个想法，用 n 比上 $\rho_{n}$ 的求和即可得到有效样本量。
  
  $ESS = \frac{N}{\sum_{t = -\infty}^{\infty} \rho_t}$

  如果自相关特别大，那么 ESS 接近1，如果自相关特别小，那么ESS接近n。

我们可以通过 `az.ess(.)` 函数计算 ESS。

In [24]:
az.ess(chains)

<xarray.Dataset>
Dimensions:      ()
Data variables:
    good_chains  float64 3.995e+03
    bad_chains0  float64 2.424
    bad_chains1  float64 130.8

可以看到，
- good_chains 的有效样本数量接近4000，说明样本的自相关问题少。
- bad_chains0 的有效样本量为2.4，该结果说明该 MCMC 样本的结果非常差。
- bad_chains1 的结果虽然比 bad_chains0 多，但是仍然很少。

一般建议 ESS 需要大于400，现在更多的要求是大于1000。

除了 ESS 外，另一个反应自相关的指标是 MCMC error，也叫 MCSE。

反映了 MCMC链的变异(方差)。其中，自相关越大，变异也越大。

我们可以通过 `az.mcse(.)` 函数计算 MCSE。

In [25]:
az.mcse(chains)

<xarray.Dataset>
Dimensions:      ()
Data variables:
    good_chains  float64 0.002576
    bad_chains0  float64 0.11
    bad_chains1  float64 0.01672

结果显示：good_chains 的 MCSE最小，bad_chains0的结果最差。
这与 ESS 的解释相符。

ArViz提供了更为方便的计算函数。通过对az.summary(.)函数的，可以一次性计算出ESS、Rhat和MCSE。

In [26]:
az.summary(chains, kind="diagnostics")

,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
good_chains,0.003,0.002,3995.0,3883.0,1.00
bad_chains0,0.110,0.090,2.0,11.0,2.43
bad_chains1,0.017,0.014,131.0,70.0,1.02


### 如何改进 MCMC 采样过程

通过**视觉诊断**和**相应的指标**。我们了解了如何诊断 MCMC 样本可能出现的问题。

在诊断出 MCMC 样本可能存在的问题后，下一步就是如何修复这些问题。

常见的解决方法包括：
- 增加 MCMC 样本数量。
  
  比如将 `trace = pm.sample(draws = 2000)` 中的 draws 扩大为4000及其以上。

- 增加 MCMC链的数量。
  
  比如将 `trace = pm.sample(draws = 2000, chains=2)` 中的 chains 扩大为4。

- 设置burn-in。
  
  比如设置 `trace = pm.sample(draws = 2000, tune = 1000)` 这会采样3000个样本，丢掉最开的1000个样本。
	
	这样做的目的在于避免初始值太极端而导致花费太多迭代来达到平稳分布。
	![Image Name](https://cdn.kesci.com/upload/image/rkijy7xre5.png?imageView2/0/w/300/h/640)

这些方法的原理非常简单，目的都是增加有效样本数量。

但对于本身就不能收敛的模型，增加样本数量也是没有用的。

需要注意的，MCMC诊断只是 Baysian workflow 的其中一环。

当通过诊断不能发现问题，我们需要返回 workflow中通过其他方法去发现潜在的问题，

常见问题包括：
- 由于先验设置错误导致模型无法收敛。
- 变量单位不统一导致参数过大或过小。
- 数据量太少导致参数估计不准确。
- 模型设定错误，导致模型无法收敛。
- 编程代码错误，导致模型采样出现问题。

除了今天介绍的 MCMC 诊断方法 和 修复方法外，还有很多其他的方法，我们会再之后的实战中进行介绍

其他的诊断方法：
- Rank Plots 
![Image Name](https://cdn.kesci.com/upload/image/rkina0ih68.png?imageView2/0/w/320/h/320)
- Parallel plots
- Separation plots
- Divergences (HMC特有的问题及诊断方法)

source: https://arviz-devs.github.io/arviz/examples/index.html

其他修复方法：
- 设置 `target_accept`。
- 参见 workflow中的设置先验，参数重整化，模型结构调整等。